# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe DataFramesMeta"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-28 12:06:14 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing Julia package DataFramesMeta...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [2]:
using DataFrames
using CSV
using Pipe
using Statistics
using DataFramesMeta

# Step 2. Create the DataFrame with the following values:

In [3]:
raw_data = Dict(:regiment => ["Nighthawks", "Nighthawks", "Nighthawks", "Nighthawks", "Dragoons", "Dragoons", "Dragoons", "Dragoons", "Scouts", "Scouts", "Scouts", "Scouts"],
    :company => ["1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd"],
    :name => ["Miller", "Jacobson", "Ali", "Milner", "Cooze", "Jacon", "Ryaner", "Sone", "Sloan", "Piger", "Riani", "Ali"],
    :preTestScore => [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
    :postTestScore => [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70])

Dict{Symbol, Vector{T} where T} with 5 entries:
  :company       => ["1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd", "1…
  :name          => ["Miller", "Jacobson", "Ali", "Milner", "Cooze", "Jacon", "…
  :preTestScore  => [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3]
  :postTestScore => [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]
  :regiment      => ["Nighthawks", "Nighthawks", "Nighthawks", "Nighthawks", "D…

# Step 3. Assign it to a variable called regiment.

In [4]:
regiment = DataFrame(; raw_data...)

,company,name,preTestScore,postTestScore,regiment
,String,String,Int64,Int64,String
1,1st,Miller,4,25,Nighthawks
2,1st,Jacobson,24,94,Nighthawks
3,2nd,Ali,31,57,Nighthawks
4,2nd,Milner,2,62,Nighthawks
5,1st,Cooze,3,70,Dragoons
6,1st,Jacon,4,25,Dragoons
7,2nd,Ryaner,24,94,Dragoons
8,2nd,Sone,31,57,Dragoons
9,1st,Sloan,2,62,Scouts


# Step 4. What is the mean preTestScore from the regiment Nighthawks?

In [8]:
nighthawks = filter(x -> x.regiment != "Nighthawks", regiment)
mean(nighthawks[!,:preTestScore])

9.0

# Step 5. Present general statistics by company

In [ ]:
company_group = groupby(regiment, :company)

In [10]:
combine(company_group, describe)

,company,variable,mean,min,median,max,nmissing,eltype
,String,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,1st,company,,1st,,1st,0,String
2,1st,name,,Cooze,,Sloan,0,String
3,1st,preTestScore,6.66667,2,3.5,24,0,Int64
4,1st,postTestScore,57.6667,25,66.0,94,0,Int64
5,1st,regiment,,Dragoons,,Scouts,0,String
6,2nd,company,,2nd,,2nd,0,String
7,2nd,name,,Ali,,Sone,0,String
8,2nd,preTestScore,15.5,2,13.5,31,0,Int64
9,2nd,postTestScore,67.0,57,62.0,94,0,Int64


# Step 6. What is the mean of each company's preTestScore?


In [11]:
combine(company_group, :preTestScore => mean)

,company,preTestScore_mean
,String,Float64
1,1st,6.66667
2,2nd,15.5


# Step 7. Present the mean preTestScores grouped by regiment and company

In [12]:
regiment_company_group = groupby(regiment, [:regiment, :company])
combine(regiment_company_group, :preTestScore => mean)

,regiment,company,preTestScore_mean
,String,String,Float64
1,Nighthawks,1st,14.0
2,Nighthawks,2nd,16.5
3,Dragoons,1st,3.5
4,Dragoons,2nd,27.5
5,Scouts,1st,2.5
6,Scouts,2nd,2.5


# Step 8. Present the mean preTestScores grouped by regiment and company without heirarchical indexing

In [13]:
combine(regiment_company_group, :preTestScore => mean, ungroup = false)

,regiment,company,preTestScore_mean
,String,String,Float64
1,Nighthawks,1st,14.0
,regiment,company,preTestScore_mean
,String,String,Float64
1,Scouts,2nd,2.5


# Step 10. What is the number of observations in each regiment and company


In [14]:
combine(regiment_company_group, :preTestScore => size)

,regiment,company,preTestScore_size
,String,String,Tuple…
1,Nighthawks,1st,"(2,)"
2,Nighthawks,2nd,"(2,)"
3,Dragoons,1st,"(2,)"
4,Dragoons,2nd,"(2,)"
5,Scouts,1st,"(2,)"
6,Scouts,2nd,"(2,)"


In [16]:
regiment_group = groupby(regiment, :regiment)

for group in regiment_group
    print(group)
end

4×5 SubDataFrame
 Row │ company  name      preTestScore  postTestScore  regiment   
     │ String   String    Int64         Int64          String     
─────┼────────────────────────────────────────────────────────────
   1 │ 1st      Miller               4             25  Nighthawks
   2 │ 1st      Jacobson            24             94  Nighthawks
   3 │ 2nd      Ali                 31             57  Nighthawks
   4 │ 2nd      Milner               2             62  Nighthawks4×5 SubDataFrame
 Row │ company  name    preTestScore  postTestScore  regiment 
     │ String   String  Int64         Int64          String   
─────┼────────────────────────────────────────────────────────
   1 │ 1st      Cooze              3             70  Dragoons
   2 │ 1st      Jacon              4             25  Dragoons
   3 │ 2nd      Ryaner            24             94  Dragoons
   4 │ 2nd      Sone              31             57  Dragoons4×5 SubDataFrame
 Row │ company  name    preTestScore  postTestSco